<a href="https://colab.research.google.com/github/JoseColombini/PCS5022-deeplearning/blob/main/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Gerador de CSV para submissão

- use **Y_predict** to save the outputs of your model. It will automatically fit it here to generate the csv

- Be sure you are sending the csv of the correct task

In [4]:
import numpy as np
import pandas as pd

# Carrega a base de dados a partir de seu caminho
data = np.load(dataset)
X_test = data['X_test']

# Faz palpites aleatórios
num_samples = X_test.shape[0]
random_predictions = np.random.randint(0, 2, size=num_samples)

# Gera um DataFrame no formato esperado da submissão
submission_df = pd.DataFrame({
    'ID': np.arange(1, num_samples + 1),
    'Prediction': random_predictions
})

# Salva o arquivo CSV no diretório atual
submission_df.to_csv('submission.csv', index=False)

# Task 1

In [2]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision.transforms import ToTensor

device = torch.accelerator.current_accelerator().type if torch.accelerator.is_available() else "cpu"
print(f"Using {device} device")

dataset = "dataset1.npz"
data = np.load(dataset)
X_train = data['X_train']
print(X_train.shape[1])

print(data)

class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

Using cuda device


NameError: name 'np' is not defined